<a href="https://colab.research.google.com/github/DineshRavindraRaju/Analyse_your_genome/blob/main/Analyse_your_genome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import uuid
import argparse
import subprocess
import os
import subprocess
from sys import exit
from subprocess import PIPE, run
import argparse
from os import walk
import shutil

Download the Refrence Datasets

In [20]:
!wget ftp://ftp.ensembl.org/pub/grch37/current/fasta/homo_sapiens/dna/Homo_sapiens.GRCh37.dna.primary_assembly.fa.gz
!wget ftp://ftp.ensembl.org/pub/release-102/variation/vep/homo_sapiens_vep_102_GRCh38.tar.gz

--2023-03-26 18:25:56--  ftp://ftp.ensembl.org/pub/grch37/current/fasta/homo_sapiens/dna/Homo_sapiens.GRCh37.dna.primary_assembly.fa.gz
           => ‘Homo_sapiens.GRCh37.dna.primary_assembly.fa.gz’
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.139
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.139|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/grch37/current/fasta/homo_sapiens/dna ... done.
==> SIZE Homo_sapiens.GRCh37.dna.primary_assembly.fa.gz ... 869923173
==> PASV ... done.    ==> RETR Homo_sapiens.GRCh37.dna.primary_assembly.fa.gz ... done.
Length: 869923173 (830M) (unauthoritative)

Homo_sapiens.GRCh37 100%[===================>] 829.62M  36.0MB/s    in 25s     

2023-03-26 18:26:22 (33.3 MB/s) - ‘Homo_sapiens.GRCh37.dna.primary_assembly.fa.gz’ saved [869923173]

--2023-03-26 18:26:22--  ftp://ftp.ensembl.org/pub/release-102/variation/vep/homo_sapiens_vep_102_GRCh38.ta

In [29]:
###############################################################################
# INSTALL CONDA ON GOOGLE COLAB
################################################################################
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
#sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2023-03-26 18:36:27--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.2’

Miniconda3-py37_4.8 100%[===================>]  81.12M   373MB/s    in 0.2s    

2023-03-26 18:36:27 (373 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.2’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: | / - \ | / - \ | / done

# All requested packages already installed.

installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH on

In [30]:
!conda --version


conda 4.8.2


In [31]:
import sys
sys.path

['/content',
 '/env/python',
 '/usr/lib/python39.zip',
 '/usr/lib/python3.9',
 '/usr/lib/python3.9/lib-dynload',
 '',
 '/usr/local/lib/python3.9/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.9/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/usr/local/lib/python3.7/site-packages/',
 '/usr/local/lib/python3.7/site-packages/']

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Convert 23andme and Ancestry Files to VCF

In [35]:
!conda install -c bioconda  bcftools --yes

Solving environment: - \ | / - \ | / - \ | / - \ | done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - bcftools


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-5.1          |            1_gnu          21 KB
    bcftools-1.9               |       ha228f0b_4         807 KB  bioconda
    bzip2-1.0.8                |       h7b6447c_0          78 KB
    c-ares-1.19.0              |       h5eee18b_0         118 KB
    ca-certificates-2023.01.10 |       h06a4308_0         120 KB
    certifi-2022.12.7          |   py37h06a4308_0         150 KB
    curl-7.88.1                |       h5eee18b_0          88 KB
    krb5-1.19.4         

In [36]:
!gunzip Homo_sapiens.GRCh37.dna.primary_assembly.fa.gz

gzip: Homo_sapiens.GRCh37.dna.primary_assembly.fa already exists; do you wish to overwrite (y or n)? n
	not overwritten


Converts the txt file to VCF format you can download the vcf file and upload it to VEP to analyse it 

In [37]:
!bcftools convert --tsv2vcf /content/AncestryDNA.txt -f /content/Homo_sapiens.GRCh37.dna.primary_assembly.fa -s Ancestry -Ob -o /content/AncestryDNA.bcf
!bcftools view /content/AncestryDNA.bcf > /content/AncestryDNA.vcf

Rows total: 	664430
Rows skipped: 	34280
Missing GTs: 	0
Hom RR: 	0
Het RA: 	415313
Hom AA: 	679
Het AA: 	214158


In [38]:
!pip install panno==0.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
!panno -s Ancestry -i /content/AncestryDNA.vcf -p EUR -o /content/


Parsing PGx related diplotypes ...
Annotating clinical information ...
Generating PAnno report ...

Your PAnno report has been completed and is located at /content/Ancestry.PAnno.html.

     ^ _ ^




In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
df=pd.read_csv('/content/AncestryDNA.vcf', sep="\t", comment='#',names=["CHROM","POS","ID","REF","ALT","QUAL","FILTER","INFO","FORMAT","Ancestry"])
rs_ids=list(df["ID"].head(10000))
print(rs_ids[0:100])

['rs369202065', 'rs199476136', 'rs3131972', 'rs114525117', 'rs12124819', 'rs4040617', 'rs141175086', 'rs115093905', 'rs11240777', 'rs6681049', 'rs4422948', 'rs57494724', 'rs4475691', 'rs6657440', 'rs4970461', 'rs13302982', 'rs2880024', 'rs74047407', 'rs1110052', 'rs2272756', 'rs3748597', 'rs13302957', 'rs13303106', 'rs13303010', 'rs200468119', 'rs6696281', 'rs28391282', 'rs28504611', 'rs28477686', 'rs6660139', 'rs13303118', 'rs2341354', 'rs4970414', 'rs116781904', 'rs78164078', 'rs6665000', 'rs2341362', 'rs9777703', 'rs1891910', 'rs35940137', 'rs3128117', 'rs2465126', 'rs2341365', 'rs786201005', 'rs8997', 'rs672601312', 'rs6657048', 'rs3128126', 'rs2488993', 'rs587777299', 'rs2710875', 'rs143324306', 'rs2465128', 'rs150132566', 'rs536657086', 'rs9442391', 'rs2275813', 'rs2275811', 'rs17160775', 'rs17160776', 'rs199476396', 'rs587777298', 'rs144245019', 'rs763818876', 'rs9803031', 'rs4275402', 'rs2465136', 'rs9331226', 'rs2488991', 'rs3766192', 'rs3766191', 'rs9442372', 'rs10907177', 'r

In [42]:
import requests, sys
 
df_all = pd.DataFrame()
df_alls = pd.DataFrame()

for id in rs_ids:
  server = "http://rest.ensembl.org"
  ext = f'/variation/human/{id}?'
  r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
 
 
  if not r.ok:
    r.raise_for_status()
    sys.exit()
 
  decoded = r.json()
  df_all=pd.json_normalize(decoded)
  df_alls=pd.DataFrame.from_dict(pd.concat([df_alls, df_all], sort=False))
  #print(repr(decoded))

#print(df_alls)
rs_i=df_alls[["name","most_severe_consequence"]]



Sorting out the varaints

In [153]:
#print(rs_i.head(5))
Putative_impact=rs_i.groupby('most_severe_consequence')['name'].apply(list).reset_index()
#print(Putative_impact.head(50))
word_freq ={'chromosome_number_variation':'high','exon_loss_variant':'high','frameshift_variant':'high','stop_lost':'high','stop_gained':'high','start_lost':'high','splice_acceptor_variant':'high','splice_donor_variant':'high'}
Putative_impact['impact'] = Putative_impact["most_severe_consequence"].map(word_freq)
print(Putative_impact.head(20))
#print(type(Putative_impact))

info about the High Varaints

In [174]:
print(Putative_impact.loc[Putative_impact['impact'] == 'high',:])

    most_severe_consequence  \
8   splice_acceptor_variant   
11     splice_donor_variant   
14               start_lost   
15              stop_gained   
16                stop_lost   

                                                 name impact  
8   [rs869025278, rs144750655, rs2233004, rs786201...   high  
11  [rs869312935, rs869025276, rs1281013, rs786204...   high  
14                         [rs869312825, rs137853206]   high  
15  [rs786201005, rs672601312, rs587777299, rs1886...   high  
16                         [rs387907290, rs768434408]   high  


Push it to the github